In [6]:
import json
data = []
with open("C:/Users/dhara/Downloads/fashion_products_data.ldjson", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line.strip()))
import pandas as pd
df = pd.DataFrame(data)
print(df.head())

                            uniq_id            crawl_timestamp        asin  \
0  26d41bdc1495de290bc8e6062d927729  2020-02-07 05:11:36 +0000  B07STS2W9T   
1  410c62298852e68f34c35560f2311e5a  2020-02-07 08:45:56 +0000  B07N6TD2WL   
2  52e31bb31680b0ec73de0d781a23cc0a  2020-02-06 11:09:38 +0000  B07WJ6WPN1   
3  25798d6dc43239c118452d1bee0fb088  2020-02-07 08:32:45 +0000  B07PYSF4WZ   
4  ad8a5a196d515ef09dfdaf082bdc37c4  2020-02-06 14:27:48 +0000  B082KXNM7X   

                                         product_url  \
0  https://www.amazon.in/Facon-Kalamkari-Handbloc...   
1  https://www.amazon.in/Sf-Jeans-Pantaloons-T-Sh...   
2  https://www.amazon.in/LOVISTA-Traditional-Prin...   
3  https://www.amazon.in/People-Printed-Regular-T...   
4  https://www.amazon.in/Monte-Carlo-Cotton-Colla...   

                                        product_name  \
0  LA' Facon Cotton Kalamkari Handblock Saree Blo...   
1  Sf Jeans By Pantaloons Men's Plain Slim fit T-...   
2  LOVISTA Cotton Gota Pat

In [2]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   uniq_id                        30000 non-null  object
 1   crawl_timestamp                30000 non-null  object
 2   asin                           30000 non-null  object
 3   product_url                    30000 non-null  object
 4   product_name                   30000 non-null  object
 5   image_urls__small              29998 non-null  object
 6   medium                         29998 non-null  object
 7   large                          28841 non-null  object
 8   browsenode                     29480 non-null  object
 9   brand                          21857 non-null  object
 10  sales_price                    27110 non-null  object
 11  weight                         30000 non-null  object
 12  rating                         30000 non-null  object
 13  s

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import Image, display

# Filter the required columns
df_filtered = df[['product_name', 'image_urls__small']].dropna()

# Bag of Words Vectorizer
vectorizer = CountVectorizer()
bow_vectors = vectorizer.fit_transform(df_filtered['product_name'])

# Function for Recommendations
def recommend_products(product_index, num_recommendations, vectors, dataframe):
   
    # Compute cosine similarity
    cosine_sim = cosine_similarity(vectors[product_index], vectors).flatten()
    
    # Get indices of the most similar products
    similar_indices = cosine_sim.argsort()[-(num_recommendations+1):][::-1]
    
    # Exclude the queried product itself
    similar_indices = [i for i in similar_indices if i != product_index][:num_recommendations]
    
    # Display recommended products
    print(f"Recommendations for: {dataframe.iloc[product_index]['product_name']}")
    for idx in similar_indices:
        product_name = dataframe.iloc[idx]['product_name']
        print(f"- {product_name}")
        if 'image_urls__small' in dataframe.columns and pd.notnull(dataframe.iloc[idx]['image_urls__small']):
            display(Image(url=dataframe.iloc[idx]['image_urls__small']))

# Example Usage
recommend_products(0, 3, bow_vectors, df_filtered)


Recommendations for: LA' Facon Cotton Kalamkari Handblock Saree Blouse Fabric 100 cms Black Base Dancers (Cotton)
- LA' Facon Cotton Kalamkari Handblock Saree Blouse Fabric 100 cms Black Base Dancers (Cotton)


- Cotton Kalamkari Handblock Saree Blouse/Kurti Fabric 100 cms Purple Colour - Budda Print


- Black Orange Cotton Saree/Cotton Saree With Running Blouse Piece (YELLOW)


In [4]:
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Load stopwords
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Initialize WordNet Lemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# Contraction dictionary (provided in the prompt)
contractions = {
    "I'm": "I am", "I've": "I have", "I'll": "I will", "I'd": "I would",
    "you're": "you are", "you've": "you have", "you'll": "you will", "you'd": "you would",
    "he's": "he is", "he'll": "he will", "he'd": "he would",
    "she's": "she is", "she'll": "she will", "she'd": "she would",
    "it's": "it is", "it'll": "it will", "we're": "we are", "we've": "we have",
    "we'll": "we will", "we'd": "we would", "they're": "they are", "they've": "they have",
    "they'll": "they will", "they'd": "they would", "that's": "that is", "there's": "there is",
    "here's": "here is", "who's": "who is", "what's": "what is", "where's": "where is",
    "when's": "when is", "why's": "why is", "how's": "how is", "can't": "cannot", 
    "won't": "will not", "n't": " not", "'re": " are", "'ve": " have", "'ll": " will",
    "'d": " would", "'s": " is"
}

# Function to expand contractions
def expand_contractions(text, contractions):
    pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in contractions.keys()) + r')\b')
    return pattern.sub(lambda x: contractions[x.group()], text)

# Function to clean product names
def clean_product_name(name):
    # 1. Expand contractions
    name = expand_contractions(name, contractions)
    
    # 2. Remove non-English words (retain numbers)
    name = re.sub(r'[^a-zA-Z0-9\s]', '', name)  # Keep only alphanumeric and space

    # 3. Tokenize words
    words = name.split()

    # 4. Remove stop words
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # 5. Lemmatize words to their base form
    lemmatized_words = [lemmatizer.lemmatize(word.lower()) for word in filtered_words]

    # Join back into a cleaned product name
    return " ".join(lemmatized_words)

# Apply cleaning to product_name column in the given DataFrame
df['cleaned_product_name'] = df['product_name'].apply(clean_product_name)

# Display cleaned DataFrame
print(df[['product_name', 'cleaned_product_name']])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dhara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dhara\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                            product_name  \
0      LA' Facon Cotton Kalamkari Handblock Saree Blo...   
1      Sf Jeans By Pantaloons Men's Plain Slim fit T-...   
2      LOVISTA Cotton Gota Patti Tassel Traditional P...   
3               People Men's Printed Regular fit T-Shirt   
4      Monte Carlo Grey Solid Cotton Blend Polo Colla...   
...                                                  ...   
29995        Indian Virasat Women's Rayon Anarkali Kurta   
29996  Urban Ranger by Pantaloons Men's Slim fit T-Shirt   
29997            Peter England Men's Regular fit T-Shirt   
29998  PINKY PARI Women's Embroidered Short Denim Str...   
29999  Gutsy Men's Full Sleeve All Over Printed Navy ...   

                                    cleaned_product_name  
0      la facon cotton kalamkari handblock saree blou...  
1            sf jean pantaloon men plain slim fit tshirt  
2      lovista cotton gota patti tassel traditional p...  
3                  people men printed regul

In [5]:
# Store the cleaned product names in a separate variable
cleaned_product_names = df['cleaned_product_name']

# Display the cleaned data stored in the variable
print(cleaned_product_names)


0        la facon cotton kalamkari handblock saree blou...
1              sf jean pantaloon men plain slim fit tshirt
2        lovista cotton gota patti tassel traditional p...
3                    people men printed regular fit tshirt
4        monte carlo grey solid cotton blend polo colla...
                               ...                        
29995            indian virasat woman rayon anarkali kurta
29996           urban ranger pantaloon men slim fit tshirt
29997                 peter england men regular fit tshirt
29998    pinky pari woman embroidered short denim strai...
29999            gutsy men full sleeve printed navy tshirt
Name: cleaned_product_name, Length: 30000, dtype: object


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import Image, display

# Filter the required columns
df_filtered = df[['cleaned_product_name', 'image_urls__small']].dropna()

# Bag of Words Vectorizer
vectorizer = CountVectorizer()
bow_vectors = vectorizer.fit_transform(df_filtered['cleaned_product_name'])

# Function for Recommendations
def recommend_products(product_index, num_recommendations, vectors, dataframe):
    # Compute cosine similarity
    cosine_sim = cosine_similarity(vectors[product_index], vectors).flatten()
    
    # Get indices of the most similar products
    similar_indices = cosine_sim.argsort()[-(num_recommendations+1):][::-1]
    
    # Exclude the queried product itself
    similar_indices = [i for i in similar_indices if i != product_index][:num_recommendations]
    
    # Display recommended products
    print(f"Recommendations for: {dataframe.iloc[product_index]['cleaned_product_name']}")
    for idx in similar_indices:
        product_name = dataframe.iloc[idx]['cleaned_product_name']
        print(f"- {product_name}")
        if 'image_urls__small' in dataframe.columns and pd.notnull(dataframe.iloc[idx]['image_urls__small']):
            display(Image(url=dataframe.iloc[idx]['image_urls__small']))

# Example Usage
recommend_products(0, 3, bow_vectors, df_filtered)



Recommendations for: la facon cotton kalamkari handblock saree blouse fabric 100 cm black base dancer cotton
- la facon cotton kalamkari handblock saree blouse fabric 100 cm black base dancer cotton


- cotton kalamkari handblock saree blousekurti fabric 100 cm purple colour budda print


- perfectblue cotton saree blouse piece


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import Image, display

# Filter the required columns
df_filtered = df[['cleaned_product_name', 'image_urls__small']].dropna()

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(df_filtered['cleaned_product_name'])

# Recommendation Function
def recommend_products_tfidf(product_index, num_recommendations, vectors, dataframe):
    
    # Compute cosine similarity
    cosine_sim = cosine_similarity(vectors[product_index], vectors).flatten()
    
    # Get indices of the most similar products
    similar_indices = cosine_sim.argsort()[-(num_recommendations + 1):][::-1]
    
    # Exclude the queried product itself
    similar_indices = [i for i in similar_indices if i != product_index][:num_recommendations]
    
    # Display recommendations
    print(f"Recommendations for: {dataframe.iloc[product_index]['cleaned_product_name']}")
    for idx in similar_indices:
        product_name = dataframe.iloc[idx]['cleaned_product_name']
        print(f"- {product_name}")
        if 'image_urls__small' in dataframe.columns and pd.notnull(dataframe.iloc[idx]['image_urls__small']):
            display(Image(url=dataframe.iloc[idx]['image_urls__small']))

# Example Usage
recommend_products_tfidf(0, 3, tfidf_vectors, df_filtered)


Recommendations for: la facon cotton kalamkari handblock saree blouse fabric 100 cm black base dancer cotton
- la facon cotton kalamkari handblock saree blouse fabric 100 cm black base dancer cotton


- cotton kalamkari handblock saree blousekurti fabric 100 cm purple colour budda print


- winza designer kalamkari art silk saree saree woman blouse


In [8]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import Image, display
import numpy as np

# Step 1: Preprocessing to create tokenized product names
def preprocess_product_names(dataframe):
    """Tokenizes and cleans the product names."""
    return dataframe['cleaned_product_name'].apply(lambda x: x.split())

# Tokenized product names
tokenized_names = preprocess_product_names(df)

# Step 2: Train Word2Vec Model
word2vec_model = Word2Vec(
    sentences=tokenized_names,
    vector_size=1000,   # Dimension of word vectors
    window=5,          # Context window size
    min_count=1,       # Minimum word frequency
    workers=4,         # Number of threads for training
    sg=1               # Skip-gram model (1 for Skip-gram, 0 for CBOW)
)

# Step 3: Generate Product Vectors
def get_product_vector(product_name, model):
    """Generates a vector for the entire product by averaging word vectors."""
    words = product_name.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)  # Average word vectors
    else:
        return np.zeros(model.vector_size)  # Return zero vector for unseen words

# Create product vectors
product_vectors = np.array([get_product_vector(name, word2vec_model) for name in df['cleaned_product_name']])

# Step 4: Recommendation System
def recommend_products_word2vec(product_index, num_recommendations, vectors, dataframe):
    """Recommends products based on cosine similarity of Word2Vec vectors."""
    # Compute cosine similarity
    cosine_sim = cosine_similarity([vectors[product_index]], vectors).flatten()
    
    # Get indices of the most similar products
    similar_indices = cosine_sim.argsort()[-(num_recommendations+1):][::-1]
    
    # Exclude the queried product itself
    similar_indices = [i for i in similar_indices if i != product_index][:num_recommendations]
    
    # Display recommended products
    print(f"Recommendations for: {dataframe.iloc[product_index]['product_name']}")
    for idx in similar_indices:
        product_name = dataframe.iloc[idx]['product_name']
        print(f"- {product_name}")
        if 'image_urls__small' in dataframe.columns and pd.notnull(dataframe.iloc[idx]['image_urls__small']):
            display(Image(url=dataframe.iloc[idx]['image_urls__small']))

# Example Usage
recommend_products_word2vec(0, 3, product_vectors, df)


Recommendations for: LA' Facon Cotton Kalamkari Handblock Saree Blouse Fabric 100 cms Black Base Dancers (Cotton)
- LA' Facon Cotton Kalamkari Handblock Saree Blouse Fabric 100 cms Black Base Dancers (Cotton)


- Cotton Kalamkari Handblock Saree Blouse/Kurti Fabric 100 cms Purple Colour - Budda Print


- Pal Bastralaya Bengal Soft Cotton Molmol Batik Printed Saree for Women- purple


In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack
from IPython.display import Image, display

# Filter the required columns
df_filtered = df[['product_name', 'image_urls__small']].dropna()

# Bag of Words Vectorizer
bow_vectorizer = CountVectorizer()
bow_vectors = bow_vectorizer.fit_transform(df_filtered['product_name'])

# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(df_filtered['product_name'])

# Hybrid Vector: Concatenate BoW and TF-IDF
hybrid_vectors = hstack([bow_vectors, tfidf_vectors])

# Function for Recommendations
def recommend_products_hybrid(product_index, num_recommendations, vectors, dataframe):
    # Compute cosine similarity
    cosine_sim = cosine_similarity(vectors[product_index], vectors).flatten()
    
    # Get indices of the most similar products
    similar_indices = cosine_sim.argsort()[-(num_recommendations + 1):][::-1]
    
    # Exclude the queried product itself
    similar_indices = [i for i in similar_indices if i != product_index][:num_recommendations]
    
    # Display recommended products
    print(f"Recommendations for: {dataframe.iloc[product_index]['product_name']}")
    for idx in similar_indices:
        product_name = dataframe.iloc[idx]['product_name']
        print(f"- {product_name}")
        if 'image_urls__small' in dataframe.columns and pd.notnull(dataframe.iloc[idx]['image_urls__small']):
            display(Image(url=dataframe.iloc[idx]['image_urls__small']))

# Example Usage
recommend_products_hybrid(1, 7, hybrid_vectors, df_filtered)


Recommendations for: Sf Jeans By Pantaloons Men's Plain Slim fit T-Shirt
- Sf Jeans By Pantaloons Men's Slim fit T-Shirt


- Sf Jeans By Pantaloons Men's Slim fit T-Shirt


- Sf Jeans By Pantaloons Men's Slim fit T-Shirt


- Sf Jeans By Pantaloons Men's Plain Regular Fit T-Shirt


- Sf Jeans By Pantaloons Men's Starred Slim Fit T-Shirt


- Ajile By Pantaloons Men's Plain Slim fit T-Shirt


- Ajile By Pantaloons Men's Plain Slim Fit T-Shirt


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import numpy as np
from IPython.display import Image, display

# Filter the required columns
df_filtered = df[['product_name', 'image_urls__small']].dropna()

# Tokenize product names
tokenized_names = df_filtered['product_name'].apply(lambda x: x.split())

# Train Word2Vec model on tokenized names
word2vec_model = Word2Vec(sentences=tokenized_names, vector_size=500, window=5, min_count=1, workers=5, sg=1)

# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(df_filtered['product_name'])

# Create a dictionary for TF-IDF weights
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_weights = dict(zip(tfidf_feature_names, tfidf_vectorizer.idf_))

# Function to calculate a weighted Word2Vec representation
def calculate_weighted_word2vec(sentence, model, tfidf_weights):
    vector = np.zeros(model.vector_size)
    total_weight = 0
    for word in sentence:
        if word in model.wv.key_to_index and word in tfidf_weights:
            weight = tfidf_weights[word]
            vector += model.wv[word] * weight
            total_weight += weight
    if total_weight > 0:
        vector /= total_weight
    return vector

# Compute weighted Word2Vec vectors for all products
weighted_vectors = np.array([
    calculate_weighted_word2vec(sentence, word2vec_model, tfidf_weights)
    for sentence in tokenized_names
])

# Function for recommendations
def recommend_products_hybrid(product_index, num_recommendations, vectors, dataframe):
    # Compute cosine similarity
    cosine_sim = cosine_similarity([vectors[product_index]], vectors).flatten()
    
    # Get indices of the most similar products
    similar_indices = cosine_sim.argsort()[-(num_recommendations + 1):][::-1]
    
    # Exclude the queried product itself
    similar_indices = [i for i in similar_indices if i != product_index][:num_recommendations]
    
    # Display recommended products
    print(f"Recommendations for: {dataframe.iloc[product_index]['product_name']}")
    for idx in similar_indices:
        product_name = dataframe.iloc[idx]['product_name']
        print(f"- {product_name}")
        if 'image_urls__small' in dataframe.columns and pd.notnull(dataframe.iloc[idx]['image_urls__small']):
            display(Image(url=dataframe.iloc[idx]['image_urls__small']))

# Example Usage
recommend_products_hybrid(0, 3, weighted_vectors, df_filtered)


Recommendations for: LA' Facon Cotton Kalamkari Handblock Saree Blouse Fabric 100 cms Black Base Dancers (Cotton)
- LA' Facon Cotton Kalamkari Handblock Saree Blouse Fabric 100 cms Black Base Dancers (Cotton)


- Cotton Kalamkari Handblock Saree Blouse/Kurti Fabric 100 cms Purple Colour - Budda Print


- Mufti Check 100 Percent Cotton Shirt


In [14]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML
from gensim.models import Word2Vec

# Train Word2Vec model on the tokenized product names
tokenized_names = [name.split() for name in df['cleaned_product_name']]
word2vec_model = Word2Vec(sentences=tokenized_names, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# Generate Word2Vec vectors for each product name
def get_word2vec_vector(tokens, model, vector_size=100):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)  # Average the word vectors for the product name
    else:
        return np.zeros(vector_size)  # Return a zero vector if no words match

word2vec_matrix = np.array([get_word2vec_vector(name.split(), word2vec_model) for name in df['cleaned_product_name']])

# Train TF-IDF on the cleaned product names
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(df['cleaned_product_name']).toarray()

# Reduce TF-IDF dimensionality using PCA
pca = PCA(n_components=100)  # Match Word2Vec vector size
reduced_tfidf_vectors = pca.fit_transform(tfidf_vectors)

# Combine Word2Vec and reduced TF-IDF vectors
hybrid_vectors = normalize(word2vec_matrix + reduced_tfidf_vectors)

# Recommendation function with HTML output
def recommend_products_hybrid_html(product_index, num_recommendations, vectors, dataframe):
    # Compute cosine similarity
    cosine_sim = cosine_similarity([vectors[product_index]], vectors).flatten()
    
    # Get similar product indices
    similar_indices = [
        i for i in cosine_sim.argsort()[::-1] if i != product_index
    ][:num_recommendations]

    # Generate HTML for recommendations
    recommendations_html = f"<h3>Recommendations for: {dataframe.iloc[product_index]['product_name']}</h3><ul>"
    for idx in similar_indices:
        product_name = dataframe.iloc[idx]['product_name']
        image_url = dataframe.iloc[idx]['image_urls__small']
        recommendations_html += f"<li><strong>{product_name}</strong><br><img src='{image_url}' width='150'></li>"
    recommendations_html += "</ul>"

    # Display HTML
    display(HTML(recommendations_html))

# Example Usage
recommend_products_hybrid_html(0, 3, hybrid_vectors, df)




In [28]:
print(df['brand'])

0                         LA' Facon
1                           unknown
2                           LOVISTA
3                           unknown
4                           unknown
                    ...            
29995                       unknown
29996    Urban Ranger by Pantaloons
29997                       unknown
29998                    PINKY PARI
29999                         Gutsy
Name: brand, Length: 30000, dtype: object


In [31]:
print(df[['asin', 'brand']].head(10))
print(df['brand'].isnull().sum())  # Check for missing brand values


         asin         brand
0  B07STS2W9T     LA' Facon
1  B07N6TD2WL       unknown
2  B07WJ6WPN1       LOVISTA
3  B07PYSF4WZ       unknown
4  B082KXNM7X       unknown
5  B07X5M41BJ   Forest Club
6  B07H9RCVHQ  PrintOctopus
7  B07PJ5QWZ3       unknown
8  B01D6VFN7O      Carahere
9  B07P43X53C    Peppermint
0


In [3]:
# Import necessary libraries
import json
import pandas as pd
from IPython.display import display, HTML

# Load data
data = []
file_path = "C:/Users/dhara/Downloads/fashion_products_data.ldjson"  # Adjust the path as needed
with open(file_path, encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line.strip()))

# Convert to DataFrame
df = pd.DataFrame(data)

# Ensure required columns exist
if 'asin' not in df.columns or 'brand' not in df.columns or 'medium' not in df.columns:
    print("Required columns ('asin', 'brand', 'medium') are missing in the dataset.")
else:
    # Fill missing brands with 'unknown'
    df['brand'] = df['brand'].fillna('unknown')

    # Recommendation function
    def recommend_same_brand_html(asin, num_recommendations, dataframe):
        # Get the brand for the given ASIN
        product = dataframe[dataframe['asin'] == asin]
        if product.empty:
            print(f"ASIN {asin} not found in the dataset.")
            return

        brand = product.iloc[0]['brand']

        # Filter products with the same brand
        same_brand_products = dataframe[dataframe['brand'] == brand]

        if same_brand_products.empty:
            print(f"No products found for brand: {brand}")
            return

        # Exclude the given ASIN from recommendations
        same_brand_products = same_brand_products[same_brand_products['asin'] != asin]

        # Limit to the top N recommendations
        recommendations = same_brand_products.head(num_recommendations)

        if recommendations.empty:
            print(f"No recommendations available for ASIN: {asin} under brand: {brand}")
            return

        # Generate HTML content
        html_content = f"<h3>Recommendations for Brand: {brand} (ASIN: {asin})</h3><div style='display: flex; flex-wrap: wrap;'>"
        for _, row in recommendations.iterrows():
            product_asin = row['asin']
            product_brand = row['brand']
            product_image = row['medium']
            html_content += f"""
            <div style='margin: 10px; text-align: center;'>
                <img src='{product_image}' alt='{product_asin}' style='width: 150px; height: 150px; object-fit: cover;'><br>
                <strong>Brand:</strong> {product_brand}<br>
                <strong>ASIN:</strong> {product_asin}
            </div>
            """
        html_content += "</div>"

        # Display HTML
        display(HTML(html_content))

    # Example usage
    example_asin = "B07XJJKMLC"  # Replace with a valid ASIN from your dataset
    num_recommendations = 100       # Number of recommendations
    recommend_same_brand_html(example_asin, num_recommendations, df)


In [4]:
df[df['asin'] == 'B077NF3G5N']['brand'].values[0]


'GIABELLA'

In [6]:
print(df['brand'].head(10))


0       LA' Facon
1         unknown
2         LOVISTA
3         unknown
4         unknown
5     Forest Club
6    PrintOctopus
7         unknown
8        Carahere
9      Peppermint
Name: brand, dtype: object


In [15]:
import pandas as pd

# Set Pandas option to display all rows and columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', None)  # Prevent line wrapping
pd.set_option('display.max_colwidth', None)  # Allow full column width

# Function to get the repetition of brand names
def brand_repetition(df):
    # Count the occurrences of each brand in the 'brand' column
    brand_count = df['brand'].value_counts()
    
    return brand_count

# Example usage:
brand_count = brand_repetition(df)
print(brand_count)


brand
unknown                                                                  8143
Max                                                                       504
Generic                                                                   245
BIBA                                                                      205
Mothercare                                                                156
Campus Sutra                                                              150
Soch                                                                      149
nauti nati                                                                132
Ada                                                                       127
GRITSTONES                                                                110
PrintOctopus                                                              102
Allen Solly Junior                                                         93
Columbia                                                  

In [26]:
# Check the actual column name for brands
print(df.columns)  # Verify the exact column name for brands

# Filter rows where the brand is "Max"
max_brand_asins = df[df['brand'] == "Fort Collins"]['asin']

# Print the ASINs for the brand "Max"
print(max_brand_asins)


Index(['asin', 'brand', 'medium'], dtype='object')
14404    B07YK3R1NG
Name: asin, dtype: object


In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML
import requests
from io import BytesIO
from PIL import Image
import json

# Load data
data = []
file_path = "C:/Users/dhara/Downloads/fashion_products_data.ldjson"  # Adjust the path as needed
with open(file_path, encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line.strip()))

# Assuming `product_name` column exists in the dataset
df = pd.DataFrame(data)
df = df[['asin', 'medium', 'product_name']]  # Include ASIN, image URL (medium), and product name

# Use the first 50 images for simplicity
df = df.head(50)

# Download and preprocess images
def preprocess_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert('RGB')
    img = img.resize((224, 224))  # Resize for VGG16
    img_array = img_to_array(img)
    return preprocess_input(img_array)

image_features = []
valid_asins = []

# Load VGG16 model for feature extraction
vgg16_model = VGG16(weights='imagenet', include_top=False, pooling='avg')

# Extract features
for _, row in df.iterrows():
    try:
        img_array = preprocess_image(row['medium'])
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        features = vgg16_model.predict(img_array)
        image_features.append(features.flatten())
        valid_asins.append(row['asin'])
    except Exception as e:
        print(f"Error processing image for ASIN {row['asin']}: {e}")

# Create a feature matrix
image_features = np.array(image_features)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 670ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 787ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 778ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 790ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 560ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 496ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 416ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 577ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 594ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 457ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 585ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━

In [2]:
# Recommendation function
def recommend_similar_images(asin, num_recommendations):
    """
    Recommend similar images based on ASIN using image similarity.
    
    Parameters:
        asin (str): The ASIN ID of the product to find recommendations for.
        num_recommendations (int): Number of similar recommendations to return.
    """
    if asin not in valid_asins:
        print(f"ASIN {asin} not found in the dataset.")
        return
    
    # Get the index of the ASIN in the dataset
    asin_index = valid_asins.index(asin)
    query_features = image_features[asin_index].reshape(1, -1)

    # Compute similarity scores with all images
    similarity_scores = cosine_similarity(query_features, image_features).flatten()
    
    # Exclude the given ASIN and get the most similar ASINs
    similar_indices = np.argsort(similarity_scores)[::-1]  # Sort in descending order
    similar_asins = [valid_asins[i] for i in similar_indices if valid_asins[i] != asin][:num_recommendations]
    similar_scores = [similarity_scores[i] for i in similar_indices if valid_asins[i] != asin][:num_recommendations]

    # Get the queried product details
    queried_product = df[df['asin'] == asin].iloc[0]
    queried_image = queried_product['medium']
    queried_name = queried_product['product_name']

    # HTML and CSS for better alignment
    html_content = f"""
    <style>
        .container {{
            display: flex;
            flex-direction: column;
            align-items: center;
            font-family: Arial, sans-serif;
        }}
        .queried-product, .recommendation {{
            border: 1px solid #ddd;
            border-radius: 10px;
            margin: 10px;
            padding: 15px;
            text-align: center;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
            max-width: 200px;
        }}
        .product-container {{
            display: flex;
            justify-content: center;
            gap: 20px;
            flex-wrap: wrap;
        }}
        img {{
            border-radius: 10px;
            width: 150px;
            height: 150px;
            object-fit: cover;
            margin-bottom: 10px;
        }}
        h3 {{
            text-align: center;
            color: #333;
        }}
    </style>
    <div class="container">
        <h3>Queried Product</h3>
        <div class="queried-product">
            <img src="{queried_image}" alt="{asin}">
            <strong>ASIN:</strong> {asin}<br>
            <strong>Product Name:</strong> {queried_name}
        </div>
        <h3>Recommended Products</h3>
        <div class="product-container">
    """
    
    for similar_asin, score in zip(similar_asins, similar_scores):
        product = df[df['asin'] == similar_asin].iloc[0]
        product_image = product['medium']
        product_name = product['product_name']
        html_content += f"""
        <div class="recommendation">
            <img src="{product_image}" alt="{similar_asin}">
            <strong>ASIN:</strong> {similar_asin}<br>
            <strong>Product Name:</strong> {product_name}<br>
            <strong>Similarity Score:</strong> {score:.2f}
        </div>
        """
    html_content += "</div></div>"

    # Display the results in an HTML format
    display(HTML(html_content))
     
recommend_similar_images("B07PYSF4WZ", 3)

In [3]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML
import requests
from io import BytesIO
from PIL import Image
import json

# Load data
data = []
file_path = "C:/Users/dhara/Downloads/fashion_products_data.ldjson"  # Adjust the path as needed
with open(file_path, encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line.strip()))

# Assuming product_name column exists in the dataset
df = pd.DataFrame(data)
df = df[['asin', 'medium', 'product_name']]  # Include ASIN, image URL (medium), and product name

# Use the first 50 images for simplicity
df = df.head(50)

# Download and preprocess images
def preprocess_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert('RGB')
    img = img.resize((224, 224))  # Resize for VGG16
    img_array = img_to_array(img)
    return preprocess_input(img_array)

image_features = []
valid_asins = []
image_arrays = []

# Load VGG16 model for feature extraction
vgg16_model = VGG16(weights='imagenet', include_top=False, pooling='avg')

# Extract features and store raw images for training
for _, row in df.iterrows():
    try:
        img_array = preprocess_image(row['medium'])
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        features = vgg16_model.predict(img_array)
        image_features.append(features.flatten())
        image_arrays.append(img_array[0])  # Keep the raw image data
        valid_asins.append(row['asin'])
    except Exception as e:
        print(f"Error processing image for ASIN {row['asin']}: {e}")

# Prepare data for training
image_features = np.array(image_features)
image_arrays = np.array(image_arrays)
num_classes = len(valid_asins)

# Define a Sequential model to work with the flattened features
model = Sequential([
    Input(shape=(512,)),  # Adjusted input to match VGG16 output features
    Dense(256, activation='relu'),  # Fully connected layer
    Dense(128, activation='relu'),
    Dense(50, activation='softmax')  # Output layer for classification (50 products)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model (10 epochs)
print("Training the model...")
labels = np.arange(len(valid_asins))  # Generate dummy labels for training
model.fit(image_features, labels, epochs=10, batch_size=8)


# Recommendation function
def recommend_similar_images(asin, num_recommendations):
    """
    Recommend similar images based on ASIN using image similarity.
    
    Parameters:
        asin (str): The ASIN ID of the product to find recommendations for.
        num_recommendations (int): Number of similar recommendations to return.
    """
    if asin not in valid_asins:
        print(f"ASIN {asin} not found in the dataset.")
        return
    
    # Get the index of the ASIN in the dataset
    asin_index = valid_asins.index(asin)
    query_features = image_features[asin_index].reshape(1, -1)

    # Compute similarity scores with all images
    similarity_scores = cosine_similarity(query_features, image_features).flatten()
    
    # Exclude the given ASIN and get the most similar ASINs
    similar_indices = np.argsort(similarity_scores)[::-1]  # Sort in descending order
    similar_asins = [valid_asins[i] for i in similar_indices if valid_asins[i] != asin][:num_recommendations]
    similar_scores = [similarity_scores[i] for i in similar_indices if valid_asins[i] != asin][:num_recommendations]

    # Get the queried product details
    queried_product = df[df['asin'] == asin].iloc[0]
    queried_image = queried_product['medium']
    queried_name = queried_product['product_name']

    # HTML and CSS for better alignment
    html_content = f"""
    <style>
        .container {{
            display: flex;
            flex-direction: column;
            align-items: center;
            font-family: Arial, sans-serif;
        }}
        .queried-product, .recommendation {{
            border: 1px solid #ddd;
            border-radius: 10px;
            margin: 10px;
            padding: 15px;
            text-align: center;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
            max-width: 200px;
        }}
        .product-container {{
            display: flex;
            justify-content: center;
            gap: 20px;
            flex-wrap: wrap;
        }}
        img {{
            border-radius: 10px;
            width: 150px;
            height: 150px;
            object-fit: cover;
            margin-bottom: 10px;
        }}
        h3 {{
            text-align: center;
            color: #333;
        }}
    </style>
    <div class="container">
        <h3>Queried Product</h3>
        <div class="queried-product">
            <img src="{queried_image}" alt="{asin}">
            <strong>ASIN:</strong> {asin}<br>
            <strong>Product Name:</strong> {queried_name}
        </div>
        <h3>Recommended Products</h3>
        <div class="product-container">
    """
    
    for similar_asin, score in zip(similar_asins, similar_scores):
        product = df[df['asin'] == similar_asin].iloc[0]
        product_image = product['medium']
        product_name = product['product_name']
        html_content += f"""
        <div class="recommendation">
            <img src="{product_image}" alt="{similar_asin}">
            <strong>ASIN:</strong> {similar_asin}<br>
            <strong>Product Name:</strong> {product_name}<br>
            <strong>Similarity Score:</strong> {score:.2f}
        </div>
        """
    html_content += "</div></div>"

    # Display the results in an HTML format
    display(HTML(html_content))

example_asin = "B07PYSF4WZ"  # Enter your desired ASIN ID here
num_recommendations = 3       # Set the number of recommendations to display
recommend_similar_images(example_asin, num_recommendations)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 950ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 751ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 737ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 731ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 791ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 772ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 828ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 809ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 718ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 770ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 760ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 766ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 784ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 811ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 697ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 776ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 770ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 932ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 774ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 800ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 968ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 810ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 928ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 880ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 790ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import json
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML
import requests
from PIL import Image
from io import BytesIO

# Load dataset
data = []
with open("C:/Users/dhara/Downloads/fashion_products_data.ldjson", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line.strip()))
df = pd.DataFrame(data)

# Step 1: Combine product_name and brand
df['combined_text'] = df['brand'].fillna('') + " " + df['product_name']

# Step 2: Tokenize the text for Word2Vec
tokenized_text = [str(text).split() for text in df['combined_text']]

# Step 3: Train the Word2Vec model
word2vec_model = Word2Vec(
    sentences=tokenized_text,
    vector_size=100,  # Embedding size
    window=5,         # Context window
    min_count=1,      # Minimum word frequency
    sg=1              # Use Skip-Gram
)

# Step 4: Generate vectors for products
def get_product_vector(text, model):
    """Average word vectors for a given text."""
    words = str(text).split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if not word_vectors:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

df['vector'] = df['combined_text'].apply(lambda x: get_product_vector(x, word2vec_model))

# Step 5: Recommendation function with images
def recommend_similar_products_with_images(input_asin, df, top_n=5):
    """Find products similar to the one identified by input_asin and show images."""
    if input_asin not in df['asin'].values:
        return f"ASIN {input_asin} not found in dataset."
    
    # Get the vector of the input ASIN
    input_vector = df.loc[df['asin'] == input_asin, 'vector'].values[0]
    
    # Compute cosine similarity with all other products
    similarities = cosine_similarity([input_vector], np.stack(df['vector'].values))[0]
    
    # Add similarity scores to the DataFrame
    df['similarity'] = similarities
    
    # Get the top N recommendations, excluding the input ASIN itself
    recommendations = df[df['asin'] != input_asin].sort_values('similarity', ascending=False).head(top_n)
    
    # Display recommendations with images
    html = "<h3>Recommended Products:</h3>"
    for _, row in recommendations.iterrows():
        html += f"""
        <div style="margin-bottom: 20px;">
            <p><b>ASIN:</b> {row['asin']}</p>
            <p><b>Product Name:</b> {row['product_name']}</p>
            <p><b>Brand:</b> {row['brand']}</p>
            <p><b>Similarity:</b> {row['similarity']:.2f}</p>
        """
        if pd.notna(row['medium']):
            try:
                response = requests.get(row['medium'])
                img = Image.open(BytesIO(response.content))
                html += f"<img src='{row['medium']}' alt='Product Image' style='width:150px;height:auto;'/></div>"
            except:
                html += "<p><i>Image not available</i></p></div>"
        else:
            html += "<p><i>Image not available</i></p></div>"

    display(HTML(html))

# Example: Recommend products for a specific ASIN
input_asin = "B07VB5NK3M"  # Replace with an ASIN from your dataset
recommend_similar_products_with_images(input_asin, df, top_n=5)


In [1]:
print("hello world")

hello world
